In [1]:
%pip install fastapi uvicorn nest_asyncio pyngrok transformers

  Obtaining dependency information for uvicorn from https://files.pythonhosted.org/packages/d2/e2/dc81b1bd1dcfe91735810265e9d26bc8ec5da45b4c0f6237e286819194c3/uvicorn-0.35.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyngrok from https://files.pythonhosted.org/packages/7b/81/121c612fed8c247e7703cfcc5c3a04389a575e95bacbc316c0d371eb6757/pyngrok-7.2.12-py3-none-any.whl.metadata
  Obtaining dependency information for click>=7.0 from https://files.pythonhosted.org/packages/85/32/10bb5764d90a8eee674e9dc6f4db6a0ab47c8c4d0d83c27f7c39ac415a4d/click-8.2.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/66.4 kB ? eta -:--:--
   ------------------ --------------------- 30.7/66.4 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 66.4/66.4 kB 720.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/102.2 kB ? eta -:--:--
   ---------------------------------------- 102.2/102.2 kB 5.7 MB/s eta 0:00:00
Note: you may need 


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer, CrossEncoder
import torch
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import os

# Enable FastAPI inside Colab
nest_asyncio.apply()

os.environ["NGROK_AUTHTOKEN"] = "2yxPcNMH7MsihB0YF62oRuRUn5E_5bJF4Ng3PPq4qk2Emj789"
!ngrok config add-authtoken $NGROK_AUTHTOKEN

# Initialize FastAPI app
app = FastAPI()

bi_model = SentenceTransformer("GphaHoa/academic_all_mpnet_base_v2")

# === Load Cross-encoder model ===
cross_encoder = CrossEncoder(
    "GphaHoa/academic_distilroberta_base",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# === Bi-encoder input ===
class TextInput(BaseModel):
    text: str

# === Cross-encoder input ===
class RerankInput(BaseModel):
    query: str
    passages: list[str]

# === Route: Bi-encoder /encode ===
@app.post("/encode")
def encode(input: TextInput):
    print(f"Encoding {input}...")
    embedding = bi_model.encode(input.text).tolist()
    return {"embedding": embedding}

# === Route: Cross-encoder /rerank ===
@app.post("/rerank")
def rerank(input: RerankInput):
    query = input.query
    passages = input.passages
    pairs = [(query, p) for p in passages]
    print("Reranking..")
    reranked = cross_encoder.rank(query, passages, return_documents=True)

    sorted_passages = [entry["text"] for entry in reranked]
    scores = [float(entry["score"]) for entry in reranked]
    print("Completed")
    return {
        "scores": scores,
        "sorted_passages": sorted_passages
    }

public_url = ngrok.connect(addr=8000, domain="active-optimal-polliwog.ngrok-free.app")
print("Public API URL:", public_url)

# Run server
uvicorn.run(app, host="0.0.0.0", port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-61' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

Public API URL: NgrokTunnel: "https://active-optimal-polliwog.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [458]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Encoding text='biology for highschool'...
INFO:     116.101.184.146:0 - "POST /encode HTTP/1.1" 200 OK
Reranking..
Completed
INFO:     116.101.184.146:0 - "POST /rerank HTTP/1.1" 200 OK
Encoding text='biology for highschool'...
INFO:     116.101.184.146:0 - "POST /encode HTTP/1.1" 200 OK
Reranking..
Completed
INFO:     116.101.184.146:0 - "POST /rerank HTTP/1.1" 200 OK
Encoding text='physic for elementary'...
INFO:     116.101.184.146:0 - "POST /encode HTTP/1.1" 200 OK
Reranking..
Completed
INFO:     116.101.184.146:0 - "POST /rerank HTTP/1.1" 200 OK
Encoding text='how to solve linear equation'...
INFO:     116.101.184.146:0 - "POST /encode HTTP/1.1" 200 OK
Reranking..
Completed
INFO:     116.101.184.146:0 - "POST /rerank HTTP/1.1" 200 OK
Encoding text='physics for student'...
INFO:     116.101.184.146:0 - "POST /encode HTTP/1.1" 200 OK
Encoding text='physics for student'...
INFO:     116.101.184.146:0 - "POST /encode HTTP/1.1" 200 OK
Reranking..
Completed
INFO:     116.101.184.146:0 -